# Imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from typing import Callable, Any
import json

import yaml
from muutils.errormode import ErrorMode

from maze_dataset.tokenization import (
    MazeTokenizerModular,
    TokenizerElement,
    MazeTokenizer,
    TokenizationMode,
    CoordTokenizers,
    PromptSequencers,
)

import maze_dataset.tokenization as md_tokenization

from maze_dataset import (
    VOCAB,
    VOCAB_LIST,
    VOCAB_TOKEN_TO_INDEX,
    LatticeMazeGenerators,
    MazeDataset,
    MazeDatasetConfig,
)

from maze_dataset.plotting import MazePlot

from maze_dataset.util import equal_except_adj_list_sequence

In [2]:
# magic autoreload
%load_ext autoreload
%autoreload 2

# `MazeTokenizerModular` Initialization

Most of the API for these tokenizers is contained in the `MazeTokenizerModular` class. The only time when users need to interact with the internal components of a `MazeTokenizerModular` is when initializing a non-default tokenizer.

In [3]:
mt_default: MazeTokenizerModular = MazeTokenizerModular()
mt_ctt: MazeTokenizerModular = MazeTokenizerModular.from_legacy(TokenizationMode.AOTP_CTT_indexed)
# mt_default, mt_ctt

The internal components of `MazeTokenizerModular` are all instances of `TokenizerElement`.

In [4]:
print("\n".join([str(elem) for elem in TokenizerElement.__subclasses__()]))

print('\nAOTP TokenizerElement Structure:\n')
print(mt_default.tokenizer_element_tree(abstract=True))
print(f'Default tokenizer elements:\n')
print(mt_default.tokenizer_element_tree())
print(yaml.dump(mt_default.tokenizer_element_dict()))

<class 'maze_dataset.tokenization.maze_tokenizer.CoordTokenizers._CoordTokenizer'>
<class 'maze_dataset.tokenization.maze_tokenizer.EdgeGroupings._EdgeGrouping'>
<class 'maze_dataset.tokenization.maze_tokenizer.EdgePermuters._EdgePermuter'>
<class 'maze_dataset.tokenization.maze_tokenizer.EdgeSubsets._EdgeSubset'>
<class 'maze_dataset.tokenization.maze_tokenizer.AdjListTokenizers._AdjListTokenizer'>
<class 'maze_dataset.tokenization.maze_tokenizer.TargetTokenizers._TargetTokenizer'>
<class 'maze_dataset.tokenization.maze_tokenizer.StepSizes._StepSize'>
<class 'maze_dataset.tokenization.maze_tokenizer.StepTokenizers._StepTokenizer'>
<class 'maze_dataset.tokenization.maze_tokenizer.PathTokenizers._PathTokenizer'>
<class 'maze_dataset.tokenization.maze_tokenizer.PromptSequencers._PromptSequencer'>

AOTP TokenizerElement Structure:

MazeTokenizerModular
	_PromptSequencer
		_CoordTokenizer
		_AdjListTokenizer
			_EdgeGrouping
			_EdgeSubset
			_EdgePermuter
		_TargetTokenizer
		_PathTokeniz

There are currently no other constructor methods. To construct a `MazeTokenizerModular` with other `TokenizerElement`s besides those available via `from_legacy`, the standard constructor with all parent `TokenizerElement`s in the tree must be used. Some `TokenizerElement`s also contain their own initialization arguments, most of which are `boolean`-typed. The most common arguments across all `TokenizerElement`s are named `pre`, `intra`, and `post`, which all control the option to add delimiter tokens to that part of the output. Other args are more specialized; see the class docstrings for more details.

# Vocabulary

All instances of `MazeTokenizerModular` uses a static vocabulary `VOCAB`, which is one of the main functional differences from `MazeTokenizer`. Direct access to the static vocabulary can be made through 3 constants:
- `VOCAB`
  - Extension of the `SPECIAL_TOKENS` dataclass
  - Supports direct property attribution
- `VOCAB_LIST: list[str]`
  - Contains the vocabulary in a list
  - Index of a token is its unique ID
- `VOCAB_TOKEN_TO_INDEX: dict[str, int]`
  - Inverse mapping of `VOCAB_LIST`, maps tokens to unique IDs

The following shows a visualizaiton of the first 5 elements of each constant.

In [5]:
print("`VOCAB`: type")
for i, t in enumerate(VOCAB):
    if i >= 5: break
    print(f"\tVOCAB.{t} = '{getattr(VOCAB, t)}'")
print('\t...')

print("\n`VOCAB_LIST`: list[str]")
for t in VOCAB_LIST[:5]:
    print(f"\t'{t}'")
print('\t...')
    
print("\n`VOCAB_TOKEN_TO_INDEX`: dict[str, int]")
for t in VOCAB_TOKEN_TO_INDEX:
    if VOCAB_TOKEN_TO_INDEX[t] >= 5: break
    print(f"\t'{t}':   \t{VOCAB_TOKEN_TO_INDEX[t]}")
print('\t...')


`VOCAB`: type
	VOCAB.ADJLIST_START = '<ADJLIST_START>'
	VOCAB.ADJLIST_END = '<ADJLIST_END>'
	VOCAB.TARGET_START = '<TARGET_START>'
	VOCAB.TARGET_END = '<TARGET_END>'
	VOCAB.ORIGIN_START = '<ORIGIN_START>'
	...

`VOCAB_LIST`: list[str]
	'<ADJLIST_START>'
	'<ADJLIST_END>'
	'<TARGET_START>'
	'<TARGET_END>'
	'<ORIGIN_START>'
	...

`VOCAB_TOKEN_TO_INDEX`: dict[str, int]
	'<ADJLIST_START>':   	0
	'<ADJLIST_END>':   	1
	'<TARGET_START>':   	2
	'<TARGET_END>':   	3
	'<ORIGIN_START>':   	4
	...


### Considerations of Static Vocabulary

- No more rasterized vs uniform indexing, it's all fixed as uniform now
- Fixed max grid size
  - There is now a fixed maximum maze size which is supported.
  - Unique tokens (`CoordTokenizers.UT`): 50x50
  - Coordinate tuple tokens (`CoordTokenizers.CTT`): 128x128
  - Mazes larger than these sizes are not supported
  - There should be fewer compatibility issues with tokenizers using different `max_grid_size` parameters
- Vocabulary access
  - Since maze-dataset 1.0, there is no need to pass around a tokenizer object or any data structure to access its custom vocabulary

### Refactoring your code from legacy `MazeTokenizer` and `TokenizationMode`
Since `MazeTokenizerModular` uses a static vocabulary, it is not backwards compatible with any models trained using a legacy `MazeTokenizer`. The `maze-transformer` library will be updated in vX.X.X to use `MazeTokenizerModular` by default in all settings. 

If you've manually specified a `MazeTokenizer` or `TokenizationMode` in your research code, the easiest way to refactor is using `MazeTokenizerModular.from_legacy`, which will convert a `MazeTokenizer` or `TokenizationMode` to its corresponding `MazeTokenizerModular` instance. Note that this correspondence means only that the stringification of mazes are equivalent; the encodings of strings to integer vocabulary indices are not.

In [6]:
legacy_maze_tokenizer: MazeTokenizer = TokenizationMode.AOTP_UT_uniform.to_legacy_tokenizer()
modular_tokenizer_equivalent: MazeTokenizerModular = MazeTokenizerModular.from_legacy(legacy_maze_tokenizer)
print(legacy_maze_tokenizer, '\n', modular_tokenizer_equivalent)


MazeTokenizer(tokenization_mode=<TokenizationMode.AOTP_UT_uniform: 'AOTP_UT_uniform'>, max_grid_size=None) 
 MazeTokenizerModular(prompt_sequencer=PromptSequencers.AOTP(coord_tokenizer=CoordTokenizers.UT(), adj_list_tokenizer=AdjListTokenizers.AdjListCoord(pre=False, post=True, shuffle_d0=True, edge_grouping=EdgeGroupings.Ungrouped(connection_token_ordinal=1), edge_subset=EdgeSubsets.ConnectionEdges(walls=False), edge_permuter=EdgePermuters.RandomCoords()), target_tokenizer=TargetTokenizers.Unlabeled(post=False), path_tokenizer=PathTokenizers.StepSequence(step_size=StepSizes.Singles(), step_tokenizers=(StepTokenizers.Coord(),), pre=False, intra=False, post=False)))


## `ALL_TOKENIZERS`

For the most part, all combinations of `TokenizerElement`s and their arguments will produce a valid and unique `MazeTokenizerModular`. However, it is not guaranteed that every possible `MazeTokenizerModular` that can be constructed will make practical sense or have been put through testing.

`ALL_TOKENIZERS` contains all the tested tokenizers pre-built. For research investigating many different tokenization schemes, one practical way to access them is by looping through `ALL_TOKENIZERS`. Be aware that the indexing of specific tokenizers in `ALL_TOKENIZERS` may change without notice.

In [7]:
# TODO: this cell fails because all_tokenizers is broken and takes too long. we kill it after a minute to give helpful error messages of where the long (infinite?) loop is.
from maze_dataset.tokenization.all_tokenizers import _get_all_tokenizers

# import threading
# import time
# import signal

# interrupt after a fixed time
# def send_interrupt():
# 	time.sleep(60)
# 	signal.raise_signal(signal.SIGINT)

# interrupt_thread = threading.Thread(target=send_interrupt)
# interrupt_thread.start()
# fancy_timeit_result = timeit_fancy(_get_all_tokenizers, repeats=1, do_profiling=True, get_return=True, namespace=locals())

ALL_TOKENIZERS: list[MazeTokenizerModular] = _get_all_tokenizers()
len(ALL_TOKENIZERS)

F:\KNC\maze-dataset\maze_dataset\tokenization\maze_tokenizer.py:1020: UserWarning: no validation implemented for AdjListTokenizers
  warnings.warn(


4105728

In [8]:


cfg: MazeDatasetConfig = MazeDatasetConfig(
    name="test",
    grid_n=3,
    n_mazes=1,
    maze_ctor=LatticeMazeGenerators.gen_dfs,
)
maze_dataset: MazeDataset = MazeDataset.from_config(
    cfg,
    do_download=False,
    load_local=False,
    do_generate=True,
    save_local=False,
    verbose=True,
    gen_parallel=False,
)

trying to get the dataset 'test-g3-n1-a_dfs-h50097'
generating dataset...


generating & solving mazes: 100%|██████████| 1/1 [00:00<00:00, 199.92maze/s]

Got dataset test with 1 items. output.cfg.to_fname() = 'test-g3-n1-a_dfs-h50097'


In [13]:
n_diff: int = 0

funky_tokenizers: list[md_tokenization.TokenizerElement] = [
    md_tokenization
]


for tokenizer in ALL_TOKENIZERS:
    a: list[str] = maze_dataset[0].as_tokens(tokenizer)
    b: list[str] = tokenizer.to_tokens(maze_dataset[0])
    try:
        ab_are_equal: bool = equal_except_adj_list_sequence(
            a, b,
            do_except=True,
            when_counter_mismatch=ErrorMode.WARN,
            when_len_mismatch=ErrorMode.WARN,
        )
    except ValueError as e:
        print(repr(tokenizer))
        # print(tokenizer.tokenizer_element_tree())
        print(yaml.dump(tokenizer.tokenizer_element_dict()))
        # print(json.dumps(tokenizer.serialize(), indent=2))
        print(" ".join(a))
        print(" ".join(b))
        print()
        raise e
    # if not ab_are_equal:
    #     print(tokenizer)
    #     print(" ".join(a))
    #     print(" ".join(b))
    #     print()
    #     n_diff += 1


print(f"Number of different tokenizations: {n_diff}")

c:\Users\mivan\AppData\Local\pypoetry\Cache\virtualenvs\maze-dataset-K69OfPxp-py3.11\Lib\site-packages\muutils\errormode.py:52: UserWarning: Rollouts are not the same length: 70 != 76
  warn_func(msg, category=warn_cls, source=except_from)
c:\Users\mivan\AppData\Local\pypoetry\Cache\virtualenvs\maze-dataset-K69OfPxp-py3.11\Lib\site-packages\muutils\errormode.py:52: UserWarning: Rollouts are not the same length: 71 != 65
  warn_func(msg, category=warn_cls, source=except_from)
c:\Users\mivan\AppData\Local\pypoetry\Cache\virtualenvs\maze-dataset-K69OfPxp-py3.11\Lib\site-packages\muutils\errormode.py:52: UserWarning: Rollouts are not the same length: 70 != 64
  warn_func(msg, category=warn_cls, source=except_from)
c:\Users\mivan\AppData\Local\pypoetry\Cache\virtualenvs\maze-dataset-K69OfPxp-py3.11\Lib\site-packages\muutils\errormode.py:52: UserWarning: Rollouts are not the same length: 65 != 59
  warn_func(msg, category=warn_cls, source=except_from)
c:\Users\mivan\AppData\Local\pypoetry\Ca

Other possible tokenizers which aren't in `ALL_TOKENIZERS` are not guaranteed to function. Since

In [ ]:
custom_tokenizer = MazeTokenizerModular(
    prompt_sequencer=PromptSequencers.AOP(include_target_special_tokens=False),
)

assert not custom_tokenizer.is_tested_tokenizer()  # Danger, use this tokenizer at your own risk!

### ALL_TOKENIZERS Reference

For each tokenizer, tokenizations and encodings of the below maze are logged in a DataFrame for reference.

In [ ]:
MazePlot(maze_dataset[0]).plot()
plt.show()

In [ ]:
tokenizers: pd.DataFrame = pd.DataFrame( 
    columns=["tokenizer", "tokens", "encoding", *mt_default.summary().keys()]
    )
tokenizers.tokenizer = ALL_TOKENIZERS
tokenizers.tokens = tokenizers.tokenizer.apply(lambda x: " ".join(maze_dataset[0].as_tokens(x)))
tokenizers.encoding = tokenizers.tokens.apply(mt_default.encode)
for k in mt_default.summary().keys():
    tokenizers[k] = tokenizers.tokenizer.apply(lambda x: getattr(x, k))

tokenizers